In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [2]:
# Data pipeline

transform = transforms.Compose([
    transforms.ToTensor(), # Converts images to tensors and scales pixels to range 0-1
    transforms.Normalize((0.1307,), (0.3081,)) # Shifts and scales to be centered around 0
])

In [3]:
# Load MNIST datasets

train_dataset = torchvision.datasets.MNIST(
    root = "./data", train = True, download = True, transform = transform
)

test_dataset = torchvision.datasets.MNIST(
    root = "./data", train = False, download = True, transform = transform
)

In [4]:
# Create data loaders

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1000, shuffle = False)

In [ ]:
# Create neural network

class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [6]:
# Device management and optimizer

# Check for GPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device}")

# Initialize model and move to GPU device
model = MNISTClassifier().to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Using mps


In [7]:
# Define training stage

def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train() # Puts model into train mode

    # Tracking variables
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data ,target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        # Track progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * (correct / total)
            print(f"[{batch_idx * 64}/{60000}]"
                  f"Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%")
            running_loss = 0.0

In [8]:
# Define evaluation stage 

def evaluate(model, test_loader, device):
    model.eval() # Puts model into evaluation mode

    # Tracking variables
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return 100. * (correct / total)
    

In [9]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch: {epoch + 1}")
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f"Test accuracy: {accuracy:.2f}%")


Epoch: 1
[6400/60000]Loss: 0.577 | Accuracy: 81.8%
[12800/60000]Loss: 0.161 | Accuracy: 88.5%
[19200/60000]Loss: 0.108 | Accuracy: 91.3%
[25600/60000]Loss: 0.084 | Accuracy: 92.8%
[32000/60000]Loss: 0.081 | Accuracy: 93.8%
[38400/60000]Loss: 0.072 | Accuracy: 94.4%
[44800/60000]Loss: 0.065 | Accuracy: 94.9%
[51200/60000]Loss: 0.059 | Accuracy: 95.3%
[57600/60000]Loss: 0.053 | Accuracy: 95.7%
Test accuracy: 98.22%

Epoch: 2
[6400/60000]Loss: 0.053 | Accuracy: 98.5%
[12800/60000]Loss: 0.041 | Accuracy: 98.7%
[19200/60000]Loss: 0.038 | Accuracy: 98.7%
[25600/60000]Loss: 0.043 | Accuracy: 98.6%
[32000/60000]Loss: 0.043 | Accuracy: 98.7%
[38400/60000]Loss: 0.043 | Accuracy: 98.6%
[44800/60000]Loss: 0.039 | Accuracy: 98.6%
[51200/60000]Loss: 0.039 | Accuracy: 98.7%
[57600/60000]Loss: 0.034 | Accuracy: 98.7%
Test accuracy: 98.80%

Epoch: 3
[6400/60000]Loss: 0.030 | Accuracy: 99.1%
[12800/60000]Loss: 0.027 | Accuracy: 99.2%
[19200/60000]Loss: 0.018 | Accuracy: 99.2%
[25600/60000]Loss: 0.036 |